<a href="https://colab.research.google.com/github/RyanCharouf/neural-network-challenge-2/blob/main/attrition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Part 1: Preprocessing

In [3]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [4]:
# Determine the number of unique values in each column.
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [5]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]


In [6]:
# Create a list of at least 10 column names to use as X data
X_columns = [
    'Age',
    'BusinessTravel',
    'DistanceFromHome',
    'Education',
    'EnvironmentSatisfaction',
    'HourlyRate',
    'JobInvolvement',
    'JobLevel',
    'NumCompaniesWorked',
    'OverTime'
]

# Create X_df using your selected columns
X_df = attrition_df[X_columns]

# Show the data types for X_df
X_df.dtypes


Age                         int64
BusinessTravel             object
DistanceFromHome            int64
Education                   int64
EnvironmentSatisfaction     int64
HourlyRate                  int64
JobInvolvement              int64
JobLevel                    int64
NumCompaniesWorked          int64
OverTime                   object
dtype: object

In [8]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.2, random_state=42)



In [9]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary

from sklearn.preprocessing import OneHotEncoder, StandardScaler


In [15]:
# Check if any object-type data needs encoding
object_columns = X_df.select_dtypes(include=['object']).columns
print("Object columns:", object_columns)

# Apply OneHotEncoder to categorical columns
X_encoder = OneHotEncoder(sparse=False, drop='first')  # Drop first avoids multicollinearity
encoded_features = X_encoder.fit_transform(X_df[object_columns])

# Create a DataFrame from the encoded features
encoded_df = pd.DataFrame(encoded_features, columns=X_encoder.get_feature_names_out())

# Combine with other numerical columns
X_df_encoded = pd.concat([X_df.drop(object_columns, axis=1), encoded_df], axis=1)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_df_encoded, y_df, test_size=0.2, random_state=42)


Object columns: Index(['BusinessTravel', 'OverTime'], dtype='object')


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [16]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
X_train_scaled = scaler.fit_transform(X_train)

# Scale the training and testing data
X_test_scaled = scaler.transform(X_test)

In [17]:
# Create a OneHotEncoder for the Department column
department_encoder = OneHotEncoder(sparse=False)

# Fit the encoder to the training data
department_encoder.fit(y_train[['Department']])


# Create two new variables by applying the encoder
# to the training and testing data
y_train_dept_encoded = department_encoder.transform(y_train[['Department']])
y_test_dept_encoded = department_encoder.transform(y_test[['Department']])



/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [18]:
# Create a OneHotEncoder for the Attrition column
attrition_encoder = OneHotEncoder(sparse=False)

# Fit the encoder to the training data
attrition_encoder.fit(y_train[['Attrition']])

# Create two new variables by applying the encoder
# to the training and testing data
y_train_attrition_encoded = attrition_encoder.transform(y_train[['Attrition']])
y_test_attrition_encoded = attrition_encoder.transform(y_test[['Attrition']])


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


## Create, Compile, and Train the Model

In [19]:
# Find the number of columns in the X training data
num_features = X_train_scaled.shape[1]


# Create the input layer
input_layer = layers.Input(shape=(num_features,))


# Create at least two shared layers
shared_layer1 = layers.Dense(16, activation='relu')(input_layer)
shared_layer2 = layers.Dense(16, activation='relu')(shared_layer1)


In [20]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
dept_hidden_layer = layers.Dense(8, activation='relu')(shared_layer2)


# Create the output layer
dept_output_layer = layers.Dense(
    y_train_dept_encoded.shape[1], activation='softmax'
)(dept_hidden_layer)

In [21]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attrition_hidden_layer = layers.Dense(8, activation='relu')(shared_layer2)


# Create the output layer
attrition_output_layer = layers.Dense(
    y_train_attrition_encoded.shape[1], activation='softmax'
)(attrition_hidden_layer)


In [22]:
# Create the model
model = Model(
    inputs=input_layer,
    outputs=[dept_output_layer, attrition_output_layer]
)


# Compile the model
model.compile(
    loss=['categorical_crossentropy', 'categorical_crossentropy'],
    optimizer='adam',
    metrics=['accuracy']
)


# Summarize the model
model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 11)]                 0         []                            
                                                                                                  
 dense (Dense)               (None, 16)                   192       ['input_1[0][0]']             
                                                                                                  
 dense_1 (Dense)             (None, 16)                   272       ['dense[0][0]']               
                                                                                                  
 dense_2 (Dense)             (None, 8)                    136       ['dense_1[0][0]']             
                                                                                              

In [23]:
# Train the model
model.fit(
    X_train_scaled,
    [y_train_dept_encoded, y_train_attrition_encoded],
    epochs=20,
    validation_split=0.2,
    batch_size=32
)



Epoch 1/20
30/30 [==============================] - 7s 69ms/step - loss: 1.8566 - dense_3_loss: 0.9157 - dense_5_loss: 0.9409 - dense_3_accuracy: 0.6543 - dense_5_accuracy: 0.2894 - val_loss: 1.6502 - val_dense_3_loss: 0.9018 - val_dense_5_loss: 0.7484 - val_dense_3_accuracy: 0.6314 - val_dense_5_accuracy: 0.4619
Epoch 2/20
30/30 [==============================] - 0s 11ms/step - loss: 1.5312 - dense_3_loss: 0.8692 - dense_5_loss: 0.6621 - dense_3_accuracy: 0.6553 - dense_5_accuracy: 0.6489 - val_loss: 1.4639 - val_dense_3_loss: 0.8557 - val_dense_5_loss: 0.6082 - val_dense_3_accuracy: 0.6314 - val_dense_5_accuracy: 0.7203
Epoch 3/20
30/30 [==============================] - 0s 11ms/step - loss: 1.3608 - dense_3_loss: 0.8203 - dense_5_loss: 0.5406 - dense_3_accuracy: 0.6553 - dense_5_accuracy: 0.8043 - val_loss: 1.3571 - val_dense_3_loss: 0.8242 - val_dense_5_loss: 0.5329 - val_dense_3_accuracy: 0.6314 - val_dense_5_accuracy: 0.7966
Epoch 4/20
30/30 [==============================] - 0s 

In [24]:
# Evaluate the model with the testing data
results = model.evaluate(
    X_test_scaled,
    [y_test_dept_encoded, y_test_attrition_encoded]
)


10/10 [==============================] - 0s 6ms/step - loss: 1.1470 - dense_3_loss: 0.7879 - dense_5_loss: 0.3592 - dense_3_accuracy: 0.6667 - dense_5_accuracy: 0.8707


In [25]:
# Print the accuracy for both department and attrition
print("Department accuracy:", results[3])
print("Attrition accuracy:", results[4])


Department accuracy: 0.6666666865348816
Attrition accuracy: 0.8707482814788818


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. Accuracy can be useful, but it doesn't take into account for class imbalance or incorrect predictions that might impact certain classes more. Some other metrics I could use are: precision, recall, F1-score, or confusion matrices to gain a deeper understanding of the model's performance.

2. I used softmax for both outputs because both outputs represent categorical data. The softmax function gives probabilities for each class, allowing for a straightforward interpretation of the model's predictions.

3. Some ways we can improve this model is:
  - Regularization: Applying dropout or L2 regularization to reduce overfitting.
  - Data Augmentation: Enhancing the dataset with additional synthetic data.
  - Feature Engineering: Creating new features or combining existing ones to improve the model's understanding.
  - Hyperparameter Tuning: Adjusting parameters like learning rate, batch size, or layer structures for optimal performance.
  - Cross-Validation: Using k-fold cross-validation to better estimate model generalization.
  - Balancing Classes: If there's class imbalance, using techniques like SMOTE or undersampling to address this.